In [2]:
import os
import librosa
import numpy as np
import soundfile as sf
import tensorflow as tf
from tensorflow.keras import layers, models

In [5]:
# Paths
base_path = os.path.abspath("..")
main_project_path = os.path.join(base_path, 'Main_Project')
raw_sick_cow_path = os.path.join(main_project_path, 'RAW_Sick_Cow')
noise_path = os.path.join(main_project_path, 'Noise')
output_path = os.path.join(main_project_path, 'noise2noise_Denoised_Sick_Cow')

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)


In [8]:
# Hyperparameters
sr = 22050  # Sample rate for audio
epochs = 50
batch_size = 16
max_len = 10 * sr  # Example: 10 seconds of audio

# Data normalization function
def normalize_audio(audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))


# 1. Data Loading and Preprocessing

def load_wav_files(path):
    files = []
    for file_name in os.listdir(path):
        if file_name.endswith('.wav'):
            wav_path = os.path.join(path, file_name)
            wav_data, _ = librosa.load(wav_path, sr=sr)
            files.append(wav_data)
    return files

# Padding or truncating function
def pad_or_truncate(audio, max_len):
    if len(audio) > max_len:
        return audio[:max_len]
    elif len(audio) < max_len:
        return np.pad(audio, (0, max_len - len(audio)), mode='constant')
    else:
        return audio

# Load and preprocess all data
raw_sick_cow_files = load_wav_files(raw_sick_cow_path)
noise_files = load_wav_files(noise_path)

# Apply padding/truncating and normalization
raw_sick_cow_data = np.array([pad_or_truncate(normalize_audio(audio), max_len) for audio in raw_sick_cow_files])
noise_data = np.array([pad_or_truncate(normalize_audio(audio), max_len) for audio in noise_files])



/var/folders/zb/pmsl1zpx7sg1hqhm44bmnf9w0000gn/T/ipykernel_32671/218431003.py:9: RuntimeWarning: invalid value encountered in divide
  return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))


In [9]:
# 2. Model Definition

def build_noise2noise_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(64, 9, padding='same', input_shape=input_shape, activation='relu'))
    model.add(layers.Conv1D(64, 9, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1D(128, 9, padding='same', activation='relu'))
    model.add(layers.Conv1D(128, 9, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1D(256, 9, padding='same', activation='relu'))
    model.add(layers.Conv1D(256, 9, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D(2))

    model.add(layers.Conv1DTranspose(256, 9, padding='same', activation='relu'))
    model.add(layers.Conv1DTranspose(256, 9, padding='same', activation='relu'))
    model.add(layers.UpSampling1D(2))

    model.add(layers.Conv1DTranspose(128, 9, padding='same', activation='relu'))
    model.add(layers.Conv1DTranspose(128, 9, padding='same', activation='relu'))
    model.add(layers.UpSampling1D(2))

    model.add(layers.Conv1DTranspose(64, 9, padding='same', activation='relu'))
    model.add(layers.Conv1DTranspose(64, 9, padding='same', activation='relu'))
    model.add(layers.UpSampling1D(2))

    model.add(layers.Conv1D(1, 9, padding='same', activation='sigmoid'))
    
    return model

# Build the model
input_shape = (None, 1)  # (time_steps, channels)
model = build_noise2noise_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# 4. Denoising and Saving Output

def denoise_audio(model, noisy_audio):
    noisy_audio = noisy_audio[np.newaxis, ..., np.newaxis]  # Reshape for model input
    denoised_audio = model.predict(noisy_audio)
    return denoised_audio[0, :, 0]  # Reshape back to 1D

# Process and save each file
for i, file_name in enumerate(os.listdir(raw_sick_cow_path)):
    if file_name.endswith('.wav'):
        noisy_wav_path = os.path.join(raw_sick_cow_path, file_name)
        noisy_audio, _ = librosa.load(noisy_wav_path, sr=sr)
        
        denoised_audio = denoise_audio(model, noisy_audio)
        denoised_audio = denoised_audio * 32767  # Revert normalization if necessary
        denoised_audio = denoised_audio.astype(np.int16)
        
        output_wav_path = os.path.join(output_path, file_name)
        sf.write(output_wav_path, denoised_audio, sr)

print("Denoising complete. Denoised files saved in", output_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Denoising complete. Denoised files saved in /Users/nathbobs/Documents/Aidall Internship/Project_Cow/Main_Project/noise2noise_Denoised_Sick_Cow
